In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define the hyperparameters
batch_size = 64
learning_rate = 0.1
epochs = 5

# Define the device on which to run the code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.linear(x)
        return out

# Define the training and testing datasets
train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor())

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model and move it to the device
model = LogisticRegression(input_size=784, output_size=10).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Train the model in a federated manner
for epoch in range(epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move the data and target to the device
        data, target = data.view(-1, 784).to(device), target.to(device)

        # Perform the local training step
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Print the training loss at the end of each epoch
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, loss.item()))

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Move the data and target to the device
            data, target = data.view(-1, 784).to(device), target.to(device)

            # Perform the forward pass and compute the loss
            output = model(data)
            test_loss += criterion(output, target).item()

            # Compute the accuracy
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Print the test loss and accuracy at the end of each epoch
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('Test Loss: {:.6f} \tTest Accuracy: {:.2f}%\n'.format(test_loss, accuracy))


100.0%


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100.0%


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



97.4%

100.0%


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100.0%


Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

Epoch: 1 	Training Loss: 0.317003
Test Loss: 0.005253 	Test Accuracy: 90.96%

Epoch: 2 	Training Loss: 0.156564
Test Loss: 0.004830 	Test Accuracy: 91.46%

Epoch: 3 	Training Loss: 0.780051
Test Loss: 0.004647 	Test Accuracy: 91.67%

Epoch: 4 	Training Loss: 0.379290
Test Loss: 0.004535 	Test Accuracy: 92.01%

Epoch: 5 	Training Loss: 0.323931
Test Loss: 0.004442 	Test Accuracy: 92.00%



In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(784, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.linear(x)
        return x

# Define the training function
def train(model, train_loader, criterion, optimizer):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Define the testing function
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += nn.functional.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

# Set the random seed for reproducibility
torch.manual_seed(0)

# Define the hyperparameters
batch_size = 32
lr = 0.1
num_epochs = 10
num_workers = 2

# Load the MNIST dataset
train_set = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_set = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

# Split the dataset into num_workers subsets
train_loaders = [torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True) for _ in range(num_workers)]

# Initialize the models and optimizers for each worker
models = [LogisticRegression() for _ in range(num_workers)]
optimizers = [optim.SGD(model.parameters(), lr=lr) for model in models]

# Train the models in a federated way
for epoch in range(num_epochs):
    for i in range(num_workers):
        train(models[i], train_loaders[i], nn.CrossEntropyLoss(), optimizers[i])

    # Aggregate the model parameters using simple averaging
    with torch.no_grad():
        for param in models[0].parameters():
            param.zero_()
            for i in range(num_workers):
                param.add_(models[i].get_parameter(param).data)
            param.div_(num_workers)

    # Use the averaged model for testing
    test_loss, accuracy = test(models[0], torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True))
    print('Epoch {}, Test set: Average loss: {:.4f}, Accuracy: {:.0f}%'.format(epoch, test_loss, accuracy))


AttributeError: 'Parameter' object has no attribute 'rpartition'